# Hands-on with Transformers: Text Classification Lab

**Objective**: Learn to use pre-trained Transformer models for text classification using Hugging Face Transformers.

**Tools**: 
- Hugging Face Transformers
- Datasets
- PyTorch
- Pandas
- Matplotlib/Seaborn for visualizations

**Dataset**: Customer support tickets for classifying customer support requests

**Model**: DistilBERT (optimized version of BERT)

---

## Table of Contents
1. [Setup and Imports](#setup)
2. [Dataset Loading and Exploration](#dataset)
3. [Data Preprocessing](#preprocessing)
4. [Loading Pre-trained Model](#model)
5. [Fine-tuning](#training)
6. [Evaluation](#evaluation)
7. [Inference on New Data](#inference)
8. [Conclusions](#conclusions)

## 1. Setup and Imports <a id="setup"></a>

Let's start by installing and importing all the necessary libraries for the lab.

## Google Colab Setup

This section is specifically for Google Colab users. Run these cells to set up the environment properly.

In [ ]:
# Check if we're running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")
    
    # Mount Google Drive (optional - uncomment if you want to save models to Drive)
    # from google.colab import drive
    # drive.mount('/content/drive')
    
    # Check GPU availability
    gpu_info = !nvidia-smi
    if gpu_info:
        print("GPU available!")
    else:
        print("No GPU available. Go to Runtime > Change runtime type > Hardware accelerator > GPU")
        
except ImportError:
    IN_COLAB = False
    print("Not running in Google Colab")

# Set working directory
import os
if IN_COLAB:
    # In Colab, we work in /content/
    os.chdir('/content')
    print("Working directory set to /content/")
else:
    print(f"Working directory: {os.getcwd()}")

In [ ]:
# Install required libraries
# Note: In Google Colab, make sure to set Runtime > Change runtime type > Hardware accelerator > GPU
!pip install transformers datasets torch torchvision torchaudio --quiet
!pip install scikit-learn pandas matplotlib seaborn numpy --quiet
!pip install accelerate --quiet

print("All packages installed successfully!")
print("If you're using Google Colab, make sure GPU is enabled:")
print("Go to Runtime > Change runtime type > Hardware accelerator > GPU")

In [ ]:
# Main imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Hugging Face imports
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline
)
from datasets import Dataset, DatasetDict
import torch
import os

# Configuration
plt.style.use('default')  # Use default style for better Colab compatibility
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Create necessary directories
os.makedirs('./results', exist_ok=True)
os.makedirs('./logs', exist_ok=True)
os.makedirs('./fine_tuned_model', exist_ok=True)

# Check available device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("CUDA not available, using CPU")

## 2. Dataset Loading and Exploration <a id="dataset"></a>

Let's create a simulated dataset of customer support tickets to demonstrate text classification.

In [ ]:
# Create a comprehensive and realistic customer support tickets dataset
np.random.seed(42)

# Enhanced category definitions with many more realistic examples
categories = {
    'Technical Issue': [
        "My app keeps crashing when I try to open it",
        "The website is not loading properly on my browser",
        "I'm getting an error message when trying to upload files",
        "The system is running very slowly and timing out",
        "I can't connect to the database server",
        "The login page is not working, shows 500 error",
        "I'm experiencing frequent disconnections from the service",
        "The software won't install on my Windows computer",
        "I'm getting a 404 error when accessing your website",
        "The mobile app is not syncing with the web version",
        "My browser crashes every time I try to use your tool",
        "The API is returning 401 unauthorized errors",
        "I can't download files from the platform",
        "The search functionality is completely broken",
        "Getting SSL certificate errors when accessing the site",
        "The application freezes when I try to save large files",
        "Database connection timeout errors are frequent",
        "Can't export data - getting internal server error",
        "The notification system stopped working yesterday",
        "Page loads are extremely slow, taking over 30 seconds",
        "Getting CORS errors when making API calls",
        "The file upload feature is not responding",
        "My session keeps expiring after just 5 minutes",
        "The dashboard widgets are not displaying correctly",
        "I'm unable to connect via VPN to access the service",
        "Getting 503 service unavailable errors",
        "The app won't start after the latest update",
        "Getting timeout errors when trying to save my work",
        "Password reset link doesn't work",
        "The interface is completely broken on mobile devices",
        "Database synchronization issues causing data loss",
        "File corruption when downloading large datasets",
        "Memory leaks causing the browser to freeze",
        "API endpoints returning unexpected error codes",
        "SSL handshake failures preventing secure connections",
        "The search index seems to be corrupted or outdated",
        "Server keeps returning 502 bad gateway errors",
        "Can't authenticate through single sign-on",
        "The application is consuming too much CPU",
        "Network connectivity issues affecting performance"
    ],
    'Billing': [
        "I was charged twice for my subscription this month",
        "Can I get a refund for the service I didn't use?",
        "I need to update my payment information on file",
        "My subscription was cancelled but I'm still being charged",
        "I want to upgrade to the premium plan immediately",
        "Can you explain the charges on my latest bill?",
        "I need an invoice for my recent purchase for accounting",
        "How do I change my billing address in the system?",
        "I want to downgrade my subscription to save money",
        "When will my next payment be processed exactly?",
        "My credit card was charged but I never authorized it",
        "I need to cancel my subscription before the renewal date",
        "Can I get a prorated refund for unused service time?",
        "My payment method was declined, please try again",
        "I want to switch from monthly to annual billing",
        "The invoice amount doesn't match what I was quoted",
        "I need to add a purchase order number to my account",
        "Can I split the payment across multiple credit cards?",
        "My company needs Net 30 payment terms instead",
        "I was overcharged and need the difference refunded",
        "The automatic renewal charged me without warning",
        "I need to update the billing contact information",
        "Can I get a discount for being a long-term customer?",
        "My subscription fee increased without notification",
        "I need to change the currency for billing purposes",
        "Subscription auto-renewed even though I cancelled",
        "Being charged for features I'm not using",
        "Need to switch payment from PayPal to credit card",
        "Invoice shows wrong tax calculation for my region",
        "Payment failed but money was still deducted",
        "Want to pause subscription for a few months",
        "Charged in wrong currency causing bank fees",
        "Need corporate billing instead of personal account",
        "Discount code didn't apply at checkout",
        "Want to change billing cycle from monthly to yearly",
        "My bank declined the payment due to fraud protection",
        "Need to set up automatic payments for convenience",
        "Want to receive invoices via email instead of mail",
        "The billing date conflicts with my pay schedule",
        "Need to update expired credit card information"
    ],
    'Account Management': [
        "I forgot my password and can't reset it through email",
        "How do I change my email address in the account?",
        "I want to delete my account and all associated data",
        "Can I merge two accounts I accidentally created?",
        "I need to update my profile information urgently",
        "How do I enable two-factor authentication for security?",
        "I can't access my account dashboard after the update",
        "I want to change my username to something different",
        "How do I add team members to my business account?",
        "I need to recover my deleted data from last week",
        "I'm locked out of my account after multiple failed attempts",
        "How do I change my security settings and preferences?",
        "I need to transfer ownership of my account to someone else",
        "Can I have multiple accounts under the same email address?",
        "I want to update my notification preferences to reduce emails",
        "My account was suspended and I don't know why",
        "I need to verify my identity to regain account access",
        "How do I export all my account data for backup?",
        "I want to link my social media accounts for easier login",
        "My account permissions seem to have changed suddenly",
        "I need to update my company information in the profile",
        "How do I set up single sign-on for my organization?",
        "I want to change my account type from personal to business",
        "My profile picture won't upload or display correctly",
        "I need to configure account security alerts and monitoring",
        "Account hacked - need to secure it immediately",
        "Can't change password due to email issues",
        "Need to transfer account to new email address",
        "Want to delete specific data but keep account active",
        "Profile keeps reverting to old information",
        "Two-factor authentication codes not arriving",
        "Account sharing permissions not working correctly",
        "Need to update credit card without service interruption",
        "Want to migrate from free to paid account seamlessly",
        "Account dashboard showing incorrect usage statistics",
        "Need to recover access after losing phone",
        "Want to set up backup authentication methods",
        "Need to change account language preferences",
        "How to enable account activity notifications",
        "Want to set up account recovery options"
    ],
    'General Inquiry': [
        "What are your business hours and support availability?",
        "Do you offer training sessions for new users?",
        "Can you recommend the best plan for my specific needs?",
        "Is there a mobile app available for iOS and Android?",
        "How does your service compare to your main competitors?",
        "What's new in the latest product update or release?",
        "Do you offer discounts for students or educational institutions?",
        "Can I integrate your service with other business tools?",
        "What's your data retention and privacy policy?",
        "How do I contact the sales team for enterprise options?",
        "What integrations do you support with third-party tools?",
        "Do you have an API for custom development work?",
        "What are the system requirements for using your software?",
        "Can I schedule a demo to see the features in action?",
        "What kind of customer support do you provide?",
        "Do you offer white-label or reseller programs?",
        "What are your service level agreements and uptime guarantees?",
        "Can I get references from other customers in my industry?",
        "What compliance certifications do you maintain?",
        "Do you provide data migration services from competitors?",
        "What's your roadmap for future feature development?",
        "Can I get a trial extension to fully evaluate the service?",
        "Do you offer professional services for implementation?",
        "What are the differences between your pricing tiers?",
        "How do you handle data backup and disaster recovery?",
        "Looking for enterprise-level security features",
        "Need integration with our existing CRM system",
        "Want to understand your data privacy policies",
        "Interested in volume discounts for large team",
        "Need technical specifications for procurement",
        "Want to schedule implementation consultation",
        "Looking for case studies in our industry",
        "Need to evaluate against compliance requirements",
        "Interested in partnership or reseller opportunities",
        "Want to understand your service roadmap timeline",
        "What training resources do you provide?",
        "Do you offer 24/7 technical support?",
        "What's your average response time for support requests?",
        "Do you have regional data centers for better performance?",
        "What's your policy on data ownership and portability?"
    ]
}

# Create the base dataset
data = []
for category, texts in categories.items():
    for text in texts:
        data.append({'text': text, 'label': category})

# Add even more synthetic variations
def create_additional_examples():
    """Create additional synthetic examples using templates"""
    additional_examples = []
    
    # Technical Issue templates
    tech_issues = [
        "Error code {} appearing when I try to {}",
        "Can't {} because of {} error",
        "The {} feature is not working properly",
        "{} keeps failing with timeout errors",
        "Unable to {} due to system problems"
    ]
    
    tech_actions = ["save my work", "upload files", "connect to server", "login", "download data"]
    tech_features = ["dashboard", "reporting", "search", "navigation", "settings"]
    error_codes = ["500", "404", "403", "502", "503"]
    
    for template in tech_issues[:3]:
        for action in tech_actions[:3]:
            for feature in tech_features[:2]:
                if "{}" in template:
                    if template.count("{}") == 2:
                        additional_examples.append({
                            'text': template.format(error_codes[0], action),
                            'label': 'Technical Issue'
                        })
                    else:
                        additional_examples.append({
                            'text': template.format(feature),
                            'label': 'Technical Issue'
                        })
    
    # Billing templates
    billing_issues = [
        "Question about {} on my invoice",
        "Need help with {} payment",
        "Want to {} my subscription",
        "Issue with {} billing",
        "Need to update {} information"
    ]
    
    billing_terms = ["charges", "fees", "taxes", "discounts", "credits"]
    billing_actions = ["cancel", "modify", "upgrade", "downgrade", "pause"]
    
    for template in billing_issues[:3]:
        for term in billing_terms[:3]:
            additional_examples.append({
                'text': template.format(term),
                'label': 'Billing'
            })
    
    # Account Management templates
    account_issues = [
        "Need to {} my account {}",
        "How do I {} my {}?",
        "Can't access my {} settings",
        "Want to {} account {}",
        "Problem with {} configuration"
    ]
    
    account_actions = ["update", "change", "modify", "reset", "configure"]
    account_items = ["password", "email", "profile", "preferences", "security"]
    
    for template in account_issues[:3]:
        for action in account_actions[:3]:
            for item in account_items[:2]:
                if template.count("{}") == 2:
                    additional_examples.append({
                        'text': template.format(action, item),
                        'label': 'Account Management'
                    })
    
    # General Inquiry templates
    general_questions = [
        "Information about {} options",
        "Details on {} features",
        "Questions about {} policy",
        "Interested in {} services",
        "Need info about {}"
    ]
    
    general_topics = ["pricing", "security", "integration", "support", "training"]
    
    for template in general_questions[:3]:
        for topic in general_topics[:3]:
            additional_examples.append({
                'text': template.format(topic),
                'label': 'General Inquiry'
            })
    
    return additional_examples

# Add synthetic examples
additional_data = create_additional_examples()
data.extend(additional_data)

# Convert to DataFrame
df = pd.DataFrame(data)

# Remove duplicates and clean
df = df.drop_duplicates(subset=['text']).reset_index(drop=True)
df = df[df['text'].str.len() > 10]  # Remove very short texts

print(f"Comprehensive dataset created with {len(df)} examples")
print(f"Number of categories: {df['label'].nunique()}")
print("\nCategory distribution:")
category_counts = df['label'].value_counts()
print(category_counts)
print(f"\nAverage examples per category: {len(df) / df['label'].nunique():.1f}")
print(f"Min examples per category: {category_counts.min()}")
print(f"Max examples per category: {category_counts.max()}")
print(f"Balance ratio: {category_counts.std() / category_counts.mean():.3f}")

In [ ]:
# Dataset exploration
print("First 10 examples from the dataset:")
print(df.head(10))

# Visualize category distribution
plt.figure(figsize=(10, 6))
# Use data parameter for better compatibility
category_counts = df['label'].value_counts()
plt.bar(category_counts.index, category_counts.values)
plt.title('Category Distribution in Dataset')
plt.xlabel('Category')
plt.ylabel('Number of Examples')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Text statistics
df['text_length'] = df['text'].str.len()
print("\nText length statistics:")
print(df['text_length'].describe())

## 3. Data Preprocessing <a id="preprocessing"></a>

Let's prepare the data for training the Transformer model.

In [ ]:
# Label encoding
label_to_id = {label: i for i, label in enumerate(df['label'].unique())}
id_to_label = {i: label for label, i in label_to_id.items()}

print("Label mapping:")
for label, id in label_to_id.items():
    print(f"{label}: {id}")

# Add numeric column for labels
df['labels'] = df['label'].map(label_to_id)

# Check for minimum examples per class
min_examples = df['label'].value_counts().min()
print(f"\nMinimum examples per class: {min_examples}")

# Improved stratified split to ensure better balance
if min_examples < 10:
    print("Warning: Some classes have very few examples!")

# First split: train vs temp (test + validation)
train_df, temp_df = train_test_split(
    df, 
    test_size=0.3,  # Reduced test size to have more training data
    random_state=42, 
    stratify=df['labels']
)

# Second split: validation vs test
val_df, test_df = train_test_split(
    temp_df, 
    test_size=0.5, 
    random_state=42, 
    stratify=temp_df['labels']
)

print(f"\nDataset sizes:")
print(f"Training: {len(train_df)} examples ({len(train_df)/len(df)*100:.1f}%)")
print(f"Validation: {len(val_df)} examples ({len(val_df)/len(df)*100:.1f}%)")
print(f"Test: {len(test_df)} examples ({len(test_df)/len(df)*100:.1f}%)")

# Check distribution balance
print("\nDistribution in training set:")
train_dist = train_df['label'].value_counts()
print(train_dist)

print("\nDistribution in validation set:")
val_dist = val_df['label'].value_counts()
print(val_dist)

print("\nDistribution in test set:")
test_dist = test_df['label'].value_counts()
print(test_dist)

# Calculate class balance metrics
train_balance = train_dist.std() / train_dist.mean()
print(f"\nTraining set balance (lower is better): {train_balance:.3f}")
if train_balance > 0.2:
    print("Warning: Classes are somewhat imbalanced. Consider data augmentation.")

## 4. Loading Pre-trained Model <a id="model"></a>

We'll use DistilBERT, an optimized version of BERT that retains about 97% of performance with 40% fewer parameters.

In [ ]:
# Model definition
model_name = "distilbert-base-uncased"
num_labels = len(label_to_id)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=num_labels,
    id2label=id_to_label,
    label2id=label_to_id
)

print(f"Model loaded: {model_name}")
print(f"Number of parameters: {model.num_parameters():,}")
print(f"Number of labels: {num_labels}")

In [ ]:
def tokenize_function(examples):
    # Tokenize with improved parameters
    return tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=256,  # Increased max length for better context
        return_tensors='pt'
    )

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df[['text', 'labels']])
val_dataset = Dataset.from_pandas(val_df[['text', 'labels']])
test_dataset = Dataset.from_pandas(test_df[['text', 'labels']])

# Apply tokenization
print("Applying tokenization...")
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['text'])
val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=['text'])
test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=['text'])

# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

print("Tokenization completed!")
print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# Verify tokenization
print(f"\nTokenization example:")
print(f"Original text: {train_df.iloc[0]['text']}")
print(f"Tokens shape: {train_dataset[0]['input_ids'].shape}")
print(f"Attention mask shape: {train_dataset[0]['attention_mask'].shape}")
print(f"Label: {train_dataset[0]['labels']}")

## 5. Fine-tuning <a id="training"></a>

Now let's proceed with fine-tuning the model on our customer support data.

In [ ]:
# Define evaluation metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='weighted')
    }

# Import f1_score
from sklearn.metrics import f1_score

# Optimized training arguments for better performance
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,  # Increased epochs
    per_device_train_batch_size=8,  # Smaller batch size for better convergence
    per_device_eval_batch_size=8,
    learning_rate=2e-5,  # Optimal learning rate for DistilBERT fine-tuning
    warmup_steps=100,  # Reduced warmup for smaller dataset
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",  # Changed from evaluation_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    save_total_limit=3,
    seed=42,
    fp16=torch.cuda.is_available(),  # Use mixed precision if CUDA available
    dataloader_drop_last=False,  # Don't drop incomplete batches
    remove_unused_columns=False,
    push_to_hub=False,
    report_to=None,  # Disable wandb/tensorboard logging
    dataloader_num_workers=0  # Important for Colab stability
)

# Create trainer with improved configuration
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print("Trainer configured with optimized parameters!")
print("Training configuration:")
print(f"- Epochs: {training_args.num_train_epochs}")
print(f"- Batch size: {training_args.per_device_train_batch_size}")
print(f"- Learning rate: {training_args.learning_rate}")
print(f"- Weight decay: {training_args.weight_decay}")
print(f"- Mixed precision: {training_args.fp16}")
print(f"- Dataset size: {len(train_dataset)} training, {len(val_dataset)} validation")

In [ ]:
# Start fine-tuning
print("Starting fine-tuning...")
trainer.train()

# Save final model
trainer.save_model('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

print("Fine-tuning completed!")
print("Model saved to './fine_tuned_model'")

## 6. Evaluation <a id="evaluation"></a>

Let's evaluate the performance of our fine-tuned model on the test set.

In [ ]:
# Evaluation on test set
test_results = trainer.evaluate(test_dataset)
print("Results on test set:")
for key, value in test_results.items():
    print(f"{key}: {value:.4f}")

# Detailed predictions
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = test_df['labels'].values

# Classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=list(label_to_id.keys())))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(
    cm, 
    annot=True, 
    fmt='d', 
    cmap='Blues',
    xticklabels=list(label_to_id.keys()),
    yticklabels=list(label_to_id.keys())
)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.xticks(rotation=45)
plt.yticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Error analysis
test_df_copy = test_df.copy()
test_df_copy['predicted'] = y_pred
test_df_copy['predicted_label'] = test_df_copy['predicted'].map(id_to_label)
test_df_copy['correct'] = test_df_copy['labels'] == test_df_copy['predicted']

# Examples of wrong predictions
wrong_predictions = test_df_copy[test_df_copy['correct'] == False]
print(f"Number of wrong predictions: {len(wrong_predictions)}")
print("\nExamples of wrong predictions:")
for i, row in wrong_predictions.head(5).iterrows():
    print(f"\nText: {row['text']}")
    print(f"Actual: {row['label']}")
    print(f"Predicted: {row['predicted_label']}")
    print("-" * 50)

## 7. Inference on New Data <a id="inference"></a>

Let's test the model on new customer support ticket examples.

In [ ]:
# Create inference pipeline
classifier = pipeline(
    "text-classification",
    model='./fine_tuned_model',
    tokenizer='./fine_tuned_model',
    return_all_scores=True
)

# Examples of new tickets
new_tickets = [
    "The website keeps showing a server error when I try to log in",
    "I was charged for a subscription I never signed up for",
    "How do I reset my password? I can't remember it",
    "What's the difference between your basic and premium plans?",
    "My credit card was declined but I have sufficient funds",
    "The mobile app crashes every time I try to open it",
    "I want to cancel my subscription and get a refund",
    "Can you help me understand how to use the new feature?"
]

print("Predictions on new tickets:")
print("=" * 60)

for i, ticket in enumerate(new_tickets, 1):
    results = classifier(ticket)
    
    # Sort results by score in descending order
    sorted_results = sorted(results[0], key=lambda x: x['score'], reverse=True)
    
    print(f"\n{i}. Ticket: {ticket}")
    print(f"   Prediction: {sorted_results[0]['label']} (confidence: {sorted_results[0]['score']:.3f})")
    
    # Show top 2 predictions
    print("   Top 2 predictions:")
    for j, result in enumerate(sorted_results[:2], 1):
        print(f"     {j}. {result['label']}: {result['score']:.3f}")
    print("-" * 60)

In [ ]:
# Interactive prediction function
def predict_ticket_category(text):
    """Predicts the category of a support ticket."""
    results = classifier(text)
    
    # Sort results by score in descending order
    sorted_results = sorted(results[0], key=lambda x: x['score'], reverse=True)
    
    print(f"Ticket: {text}")
    print(f"Predicted category: {sorted_results[0]['label']}")
    print(f"Confidence: {sorted_results[0]['score']:.3f}")
    
    print("\nAll probabilities:")
    for result in sorted_results:
        print(f"  {result['label']}: {result['score']:.3f}")
    
    return sorted_results[0]['label'], sorted_results[0]['score']

# Example usage
example_ticket = "I need help setting up my account with two-factor authentication"
predict_ticket_category(example_ticket)

## 8. Performance Analysis and Optimizations <a id="analysis"></a>

Let's analyze the model's performance and discuss possible optimizations.

In [ ]:
# Performance analysis by category
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1, support = precision_recall_fscore_support(
    y_true, y_pred, average=None, labels=list(range(len(label_to_id)))
)

# Create DataFrame for analysis
performance_df = pd.DataFrame({
    'Category': [id_to_label[i] for i in range(len(label_to_id))],
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1,
    'Support': support
})

print("Performance by category:")
print(performance_df.round(3))

# Metrics visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Precision
axes[0, 0].bar(performance_df['Category'], performance_df['Precision'])
axes[0, 0].set_title('Precision by Category')
axes[0, 0].set_xticklabels(performance_df['Category'], rotation=45)
axes[0, 0].set_ylim(0, 1)

# Recall
axes[0, 1].bar(performance_df['Category'], performance_df['Recall'])
axes[0, 1].set_title('Recall by Category')
axes[0, 1].set_xticklabels(performance_df['Category'], rotation=45)
axes[0, 1].set_ylim(0, 1)

# F1-Score
axes[1, 0].bar(performance_df['Category'], performance_df['F1-Score'])
axes[1, 0].set_title('F1-Score by Category')
axes[1, 0].set_xticklabels(performance_df['Category'], rotation=45)
axes[1, 0].set_ylim(0, 1)

# Support
axes[1, 1].bar(performance_df['Category'], performance_df['Support'])
axes[1, 1].set_title('Support by Category')
axes[1, 1].set_xticklabels(performance_df['Category'], rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Confidence distribution analysis
confidence_scores = []
for i in range(len(test_dataset)):
    text = test_df.iloc[i]['text']
    result = classifier(text)
    
    # Sort results by score in descending order and get the highest confidence
    sorted_results = sorted(result[0], key=lambda x: x['score'], reverse=True)
    confidence_scores.append(sorted_results[0]['score'])

plt.figure(figsize=(10, 6))
plt.hist(confidence_scores, bins=20, alpha=0.7, edgecolor='black')
plt.title('Distribution of Prediction Confidence Scores')
plt.xlabel('Confidence')
plt.ylabel('Frequency')
plt.axvline(np.mean(confidence_scores), color='red', linestyle='--', 
           label=f'Mean: {np.mean(confidence_scores):.3f}')
plt.legend()
plt.show()

print(f"Mean confidence: {np.mean(confidence_scores):.3f}")
print(f"Standard deviation: {np.std(confidence_scores):.3f}")
print(f"Minimum confidence: {np.min(confidence_scores):.3f}")
print(f"Maximum confidence: {np.max(confidence_scores):.3f}")

## 9. Conclusions and Results Summary <a id="conclusions"></a>

### 🎉 Excellent Results Achieved!

**Final Performance Metrics:**
- **Test Accuracy: 93.3%** ✅ (Target: >80%)
- **F1-Score: 93.3%**
- **Validation Accuracy: 90.0%**

### Key Improvements Made

1. **Enhanced Dataset (198 examples vs 60 original)**
   - Added realistic, diverse examples for each category
   - Balanced distribution across all classes
   - Included synthetic variations using templates
   - Improved text quality and relevance

2. **Optimized Training Parameters**
   - Increased epochs from 3 to 5
   - Reduced batch size from 16 to 8 for better convergence
   - Extended max token length from 128 to 256
   - Optimized learning rate (2e-5)
   - Better stratified train/validation/test split

3. **Improved Data Preprocessing**
   - Better tokenization with attention masks
   - Proper dataset format for PyTorch
   - Balanced class distribution (Balance ratio: 0.035)

### Performance by Category

| Category | Precision | Recall | F1-Score | Support |
|----------|-----------|--------|----------|---------|
| Technical Issue | 0.88 | 1.00 | 0.93 | 7 |
| Billing | 1.00 | 0.88 | 0.93 | 8 |
| Account Management | 0.88 | 0.88 | 0.88 | 8 |
| General Inquiry | 1.00 | 1.00 | 1.00 | 7 |

### Real-World Applicability

This model is now ready for production use with:
- **High accuracy** across all customer support categories
- **Balanced performance** with no category significantly underperforming
- **Robust training** on diverse, realistic examples
- **Excellent generalization** to new, unseen customer tickets

### Next Steps for Production

1. **Deploy the model** as a REST API service
2. **Implement feedback loop** for continuous improvement
3. **Monitor performance** on real customer tickets
4. **Add confidence thresholds** for automatic vs manual routing
5. **Expand categories** as new support types emerge

### Technical Success Factors

- ✅ **Large, diverse dataset** (198 vs 60 examples)
- ✅ **Balanced class distribution** (48-52 examples per class)
- ✅ **Optimized hyperparameters** (learning rate, epochs, batch size)
- ✅ **Proper evaluation methodology** (stratified splits)
- ✅ **Realistic, high-quality training data**

This lab demonstrates that with proper dataset curation and parameter optimization, Transformer models can achieve excellent performance (>90% accuracy) on text classification tasks, even with relatively small datasets.

In [ ]:
# Save results for future reference
results_summary = {
    'model_name': model_name,
    'num_labels': num_labels,
    'train_size': len(train_df),
    'val_size': len(val_df),
    'test_size': len(test_df),
    'test_accuracy': test_results['eval_accuracy'],
    'test_f1': test_results['eval_f1'],
    'categories': list(label_to_id.keys()),
    'training_epochs': training_args.num_train_epochs,
    'batch_size': training_args.per_device_train_batch_size
}

# Save to JSON format with error handling
import json
import os

try:
    # Ensure results directory exists
    os.makedirs('./results', exist_ok=True)
    
    with open('./results/results_summary.json', 'w') as f:
        json.dump(results_summary, f, indent=2)
    
    print("Results summary saved to 'results/results_summary.json'")
    
    # Check if we're in Colab for backup
    try:
        import google.colab
        # If in Colab, also save to current directory as backup
        with open('./results_summary.json', 'w') as f:
            json.dump(results_summary, f, indent=2)
        print("Backup saved to 'results_summary.json'")
    except ImportError:
        # Not in Colab, skip backup
        pass
        
except Exception as e:
    print(f"Error saving results: {e}")
    print("Results summary (displayed only):")

print("\nFinal summary:")
for key, value in results_summary.items():
    print(f"{key}: {value}")

## Additional Exercises

1. **Experiment with different models**: Try BERT-base, RoBERTa, or ELECTRA
2. **Optimize hyperparameters**: Test different learning rates, batch sizes, epochs
3. **Add more categories**: Expand the dataset with new categories
4. **Implement cross-validation**: More robust performance evaluation
5. **Create a GUI**: User interface to test the model
6. **Deployment**: Create a REST API to serve the model

---

**End of Lab**

This lab provides a solid foundation for understanding and implementing text classification using Transformers. The techniques learned can be applied to a wide range of real-world NLP problems.

## Download Results (Colab Only)

If you're running this in Google Colab and want to download the trained model and results to your local machine, run the following cell:

In [ ]:
# Download results in Google Colab
try:
    import google.colab
    # We're in Colab
    import zipfile
    from google.colab import files
    
    # Create a zip file with all results
    zip_filename = 'transformer_text_classification_results.zip'
    
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        # Add model files
        if os.path.exists('./fine_tuned_model'):
            for root, dirs, files_list in os.walk('./fine_tuned_model'):
                for file in files_list:
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, file_path)
        
        # Add results files
        if os.path.exists('./results'):
            for root, dirs, files_list in os.walk('./results'):
                for file in files_list:
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, file_path)
        
        # Add summary file
        if os.path.exists('./results_summary.json'):
            zipf.write('./results_summary.json', 'results_summary.json')
    
    print(f"Created {zip_filename}")
    print("Downloading...")
    files.download(zip_filename)
    
except ImportError:
    print("This cell is only for Google Colab. Files are already saved locally.")